# Ajuste banco de dados 

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from collections import Counter

## Installments_payments

In [2]:
installments_payments = pd.read_csv('../../base_de_dados/installments_payments.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../base_de_dados/installments_payments.csv'

In [7]:
installments_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [8]:
#IDENTIFICA O PERCENTUAL DE MISSING DE CADA COLUNA
installments_payments_missing = installments_payments.isnull().sum().sort_values(ascending = False)/len(installments_payments)
installments_payments_missing

AMT_PAYMENT               0.000214
DAYS_ENTRY_PAYMENT        0.000214
AMT_INSTALMENT            0.000000
DAYS_INSTALMENT           0.000000
NUM_INSTALMENT_NUMBER     0.000000
NUM_INSTALMENT_VERSION    0.000000
SK_ID_CURR                0.000000
SK_ID_PREV                0.000000
dtype: float64

In [9]:
#CRIA UM DATAFRAME COM APENAS AS COLUNAS COM VALORES MISSING
installments_payments_missing = installments_payments.isnull().sum().sort_values(ascending = False)/len(installments_payments)

installments_payments_missing = pd.Series(installments_payments_missing).where(lambda x : x!=0).dropna().index.to_list()

df_installments_payments_missing = installments_payments[installments_payments_missing]

df_installments_payments_missing

,AMT_PAYMENT,DAYS_ENTRY_PAYMENT
0,6948.360,-1187.0
1,1716.525,-2156.0
2,25425.000,-63.0
3,24350.130,-2426.0
4,2160.585,-1366.0
...,...,...
13605396,NaN,NaN
13605397,NaN,NaN
13605398,NaN,NaN
13605399,NaN,NaN


In [10]:
#COLOCA A MEDIANA NOS DADOS MISSING
imputer = SimpleImputer(strategy='median')
imputer.fit(df_installments_payments_missing)

installments_payments[installments_payments_missing] = imputer.transform(df_installments_payments_missing)

In [11]:
installments_payments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,-827.0,67.500,8125.515
13605397,1310347,414406,0.0,47,-1539.0,-827.0,67.500,8125.515
13605398,1308766,402199,0.0,43,-7.0,-827.0,43737.435,8125.515
13605399,1062206,409297,0.0,43,-1986.0,-827.0,67.500,8125.515


In [12]:
#VALIDA SE TODOS OS VALORES MISSING FORAM PREENCHIDOS
installments_payments.isnull().sum().sort_values(ascending = False)/len(installments_payments)

AMT_PAYMENT               0.0
AMT_INSTALMENT            0.0
DAYS_ENTRY_PAYMENT        0.0
DAYS_INSTALMENT           0.0
NUM_INSTALMENT_NUMBER     0.0
NUM_INSTALMENT_VERSION    0.0
SK_ID_CURR                0.0
SK_ID_PREV                0.0
dtype: float64

### Criando algumas variáveis

In [13]:
installments_payments['data_pagamento'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']

In [14]:
installments_payments['valor_pago'] = installments_payments['AMT_PAYMENT'] - installments_payments['AMT_INSTALMENT']

In [15]:
installments_payments.drop(['AMT_INSTALMENT', 'AMT_PAYMENT',
                            'DAYS_INSTALMENT', 'DAYS_INSTALMENT'], axis=1, inplace=True)

In [16]:
def conta_forma_pagamentos(instancia):
    return len(np.unique(instancia))

In [17]:
# AGRUPANDO COLUNAS DA TABELA
dic = {
    'NUM_INSTALMENT_VERSION': conta_forma_pagamentos, 
    'DAYS_ENTRY_PAYMENT':np.sum,
    'data_pagamento': np.mean,
    'valor_pago': np.mean
}

installments_payments = installments_payments.groupby(['SK_ID_CURR']).agg(dic).reset_index()

In [19]:
installments_payments.to_csv('../../base_de_dados/installments_payments_filled.csv', index=False)

## POS_CASH_balance

In [20]:
POS_CASH_balance = pd.read_csv('../../base_de_dados/POS_CASH_balance.csv')

In [21]:
POS_CASH_balance = POS_CASH_balance[POS_CASH_balance['MONTHS_BALANCE'] >= -6].reset_index(drop=True)

In [22]:
POS_CASH_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048748 entries, 0 to 1048747
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   SK_ID_PREV             1048748 non-null  int64  
 1   SK_ID_CURR             1048748 non-null  int64  
 2   MONTHS_BALANCE         1048748 non-null  int64  
 3   CNT_INSTALMENT         1044573 non-null  float64
 4   CNT_INSTALMENT_FUTURE  1044573 non-null  float64
 5   NAME_CONTRACT_STATUS   1048748 non-null  object 
 6   SK_DPD                 1048748 non-null  int64  
 7   SK_DPD_DEF             1048748 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 64.0+ MB


In [23]:
#IDENTIFICA O PERCENTUAL DE MISSING DE CADA COLUNA
POS_CASH_balance_missing = POS_CASH_balance.isnull().sum().sort_values(ascending = False)/len(POS_CASH_balance)
POS_CASH_balance_missing

CNT_INSTALMENT_FUTURE    0.003981
CNT_INSTALMENT           0.003981
SK_DPD_DEF               0.000000
SK_DPD                   0.000000
NAME_CONTRACT_STATUS     0.000000
MONTHS_BALANCE           0.000000
SK_ID_CURR               0.000000
SK_ID_PREV               0.000000
dtype: float64

In [24]:
#CRIA UM DATAFRAME COM APENAS AS COLUNAS COM VALORES MISSING
POS_CASH_balance_missing = POS_CASH_balance.isnull().sum().sort_values(ascending = False)/len(POS_CASH_balance)

POS_CASH_balance_missing = pd.Series(POS_CASH_balance_missing).where(lambda x : x!=0).dropna().index.to_list()

df_POS_CASH_balance_missing = POS_CASH_balance[POS_CASH_balance_missing]

df_POS_CASH_balance_missing

,CNT_INSTALMENT_FUTURE,CNT_INSTALMENT
0,12.0,12.0
1,8.0,12.0
2,10.0,12.0
3,14.0,16.0
4,14.0,16.0
...,...,...
1048743,0.0,24.0
1048744,12.0,36.0
1048745,12.0,36.0
1048746,0.0,24.0


In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le = LabelEncoder()
POS_CASH_balance['NAME_CONTRACT_STATUS'] = le.fit_transform(POS_CASH_balance['NAME_CONTRACT_STATUS'])


In [27]:
#COLOCA A MEDIANA NOS DADOS MISSING
imputer = SimpleImputer(strategy='median')
imputer.fit(df_POS_CASH_balance_missing)

POS_CASH_balance[POS_CASH_balance_missing] = imputer.transform(df_POS_CASH_balance_missing)

In [28]:
POS_CASH_balance.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [29]:
POS_CASH_balance['parcelas_faltantes'] = POS_CASH_balance['CNT_INSTALMENT_FUTURE'] - POS_CASH_balance['CNT_INSTALMENT']

In [30]:
POS_CASH_balance['DPD'] = POS_CASH_balance['SK_DPD_DEF'] - POS_CASH_balance['SK_DPD']

In [31]:
#VALIDA SE TODOS OS VALORES MISSING FORAM PREENCHIDOS
POS_CASH_balance.isnull().sum().sort_values(ascending = False)/len(installments_payments)

DPD                      0.0
parcelas_faltantes       0.0
SK_DPD_DEF               0.0
SK_DPD                   0.0
NAME_CONTRACT_STATUS     0.0
CNT_INSTALMENT_FUTURE    0.0
CNT_INSTALMENT           0.0
MONTHS_BALANCE           0.0
SK_ID_CURR               0.0
SK_ID_PREV               0.0
dtype: float64

In [32]:
from sklearn.preprocessing import OneHotEncoder

In [33]:
ohe = OneHotEncoder(drop='first', sparse=False)

ohe.fit(POS_CASH_balance[['NAME_CONTRACT_STATUS']])

labels = ['NAME_CONTRACT_STATUS' + str(i+1) for i in range(len(ohe.categories_[0]) - 1)]

POS_CASH_balance[labels] = ohe.transform(POS_CASH_balance[['NAME_CONTRACT_STATUS']])

In [34]:
POS_CASH_balance.drop('NAME_CONTRACT_STATUS', axis=1, inplace=True)

In [35]:
POS_CASH_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,parcelas_faltantes,DPD,NAME_CONTRACT_STATUS1,NAME_CONTRACT_STATUS2,NAME_CONTRACT_STATUS3,NAME_CONTRACT_STATUS4,NAME_CONTRACT_STATUS5,NAME_CONTRACT_STATUS6,NAME_CONTRACT_STATUS7
0,2373788,146563,-1,12.0,12.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2344408,444611,-6,12.0,8.0,0,0,-4.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2235803,270883,-6,12.0,10.0,0,0,-2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1434285,450849,-2,16.0,14.0,0,0,-2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1204938,203479,-3,16.0,14.0,0,0,-2.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048743,2478461,307071,-5,24.0,0.0,0,0,-24.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1048744,2715310,390187,-6,36.0,12.0,0,0,-24.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048745,1386973,204579,-4,36.0,12.0,0,0,-24.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048746,1832464,182132,-4,24.0,0.0,0,0,-24.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [38]:
dic = {}
for label in labels:
    dic[label] = np.sum

dic['parcelas_faltantes'] = np.sum
dic['DPD'] = np.mean

In [39]:
POS_CASH_balance = POS_CASH_balance.groupby('SK_ID_CURR').agg(dic).reset_index()

In [41]:
POS_CASH_balance.to_csv('../../base_de_dados/POS_CASH_balance_filled.csv', index=False)